In [1]:
from dotenv import load_dotenv
import os
import pandas as pd
import numpy as np
from langchain_chroma import Chroma
import json
import requests

In [2]:
load_dotenv()

True

In [3]:
headers = {
    "authorization": os.getenv("myserver_api_key")
}

embed_url = os.getenv("myserver_url")+":8000/embed"
rerank_url = os.getenv("myserver_url")+":8000/rerank"

In [4]:
class CustomEmbeddings:
    def embed_documents(self, texts):
        payload = {
            "texts": texts,
            "is_query": False
        }
        return requests.post(embed_url, json=payload, headers=headers).json()["embeddings"]
    
    def embed_query(self, text):
        payload = {
            "texts": [text],
            "is_query": True
        }
        return requests.post(embed_url, json=payload, headers=headers).json()["embeddings"][0]

In [5]:
embeddings = CustomEmbeddings()

In [6]:
vector_store = Chroma(
    persist_directory="../chroma_db",
    embedding_function=embeddings
)

In [7]:
# mmr 기반
retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={"k":10, "fetch_k":50}
)

In [8]:
query = "휴가"

In [9]:
docs = retriever.get_relevant_documents(query)
docs

C:\Users\user1\AppData\Local\Temp\ipykernel_2260\2675164048.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(query)


[Document(id='ad59eaa3-ec83-45d3-a9c6-c5edc5fca186', metadata={}, page_content='법정 휴가 규정'),
 Document(id='b4cefeb4-5de7-4b1c-af69-0288b7df75bf', metadata={}, page_content='CoreFlow는 근로자의 충분한 휴식을 보장하고 일과 삶의 균형을 지원하기 위해 근로기준법 및 관련 법규에 따른 휴가 제도를 운영한다'),
 Document(id='5108c9d4-89f2-4309-9328-8ec6793e2426', metadata={}, page_content='이는 부모, 배우자, 자녀 등 돌봄이 필요한 가족을 위해 사용하는 휴가로, 연간 최대 10일까지 하루 단위로 분할하여 사용할 수 있습니다'),
 Document(id='fbae571c-f7a1-4b74-b54c-54ea63f35ec2', metadata={}, page_content='정의 및 적용 범위: 재택근무는 부여받은 업무를 자택 등 지정된 장소에서 수행하는 근무 형태를 의미한다'),
 Document(id='2fd80cc1-91cf-41f2-a90c-c89bc3ce8956', metadata={}, page_content='사내 복지 시설: 사내 휴게실, 식당 등을 운영하여 편의를 제공하고, 필요에 따라 휴양 시설 등을 임차하여 직원들이 이용할 수 있도록 지원한다'),
 Document(id='8d3bc264-facb-4c6f-a09e-e427c4f1e1ce', metadata={}, page_content='5배, 휴일근로는 8시간 이내는 1'),
 Document(id='e463751c-39b8-4767-89b4-8aeadc68d527', metadata={}, page_content='조사 기간 중 피해자가 요청하는 경우, 근무 장소 변경, 유급 휴가 명령 등 피해자를 보호하기 위한 적절한 조치를 취할 수 있다'),
 Document(id='ab71e4fe-c9d9

In [10]:
for doc in docs:
    print(doc.page_content)
    break

법정 휴가 규정


In [11]:
parent_docs_df = pd.read_csv("../data/parent_split_docs_20250909.csv")
parent_docs_list = parent_docs_df["docs"].tolist()
len(parent_docs_list)

45

In [12]:
parent_docs_list[:5]

['서론 및 CoreFlow의 경영 철학.\n\n보고서 목적 및 적용 범위.\n본 보고서는 가상의 웹 애플리케이션 개발 회사인 주식회사 CoreFlow의 체계적이고 투명한 운영을 위한 사내 규정을 총망라하여 제공한다. 이 규정집은 단순히 법적 의무를 명시하는 서류철을 넘어, CoreFlow의 모든 구성원들이 회사의 비전과 가치에 따라 행동하고, 법률 및 윤리적 기준을 준수하며, 궁극적으로 회사의 지속 가능한 성장을 도모할 수 있도록 돕는 실무적 가이드라인 역할을 한다.\n사내 규정은 명확한 규칙과 절차를 확립함으로써 회사 운영의 일관성을 유지하고, 불필요한 혼란과 갈등을 줄이는 데 필수적이다. 특히 IT 산업의 급변하는 환경 속에서, 본 규정집은 인재 관리, 지식재산권 보호, 정보 보안 등 핵심적인 영역에서 발생하는 다양한 문제를 사전에 예방하고 해결하는 데 중요한 역할을 수행할 것이다. 또한, 신입 직원의 원활한 온보딩을 돕고, 모든 구성원에게 기대되는 행동 양식과 회사의 보상 및 복리후생 체계를 투명하게 제공하여 HR 프로세스를 간소화하고 유능한 인재를 유치 및 유지하는 데 기여한다.\n\nCoreFlow의 미션, 비전 및 핵심 가치.\nCoreFlow의 모든 규정과 정책은 회사의 근본적인 경영 철학에 기반한다. 단순한 문구가 아닌, 모든 의사결정과 행동의 기준이 되는 미션, 비전, 핵심 가치는 다음과 같이 선언된다.\n미션 선언문: "최고의 기술과 창의적인 솔루션으로 고객에게 가치 있는 디지털 경험을 제공합니다."\n비전 선언문: "직원, 고객, 커뮤니티가 모두 함께 성장하고 싶어하는 최고의 기술 선도 기업이 됩니다."\n핵심 가치 선언문: "Think Different, Work Together"\nCoreFlow의 경영 철학은 특정 상황에서 직원들이 복잡한 규칙을 일일이 따르기보다, 회사의 핵심 가치에 따라 스스로 판단하고 행동할 수 있도록 하는 데 중점을 둔다. 예를 들어, "Think Different"는 Apple의 유명한 슬로건처럼 창의성과 혁

In [13]:
def search_parent_docs(docs, parent_docs):
    searched_docs = []
    for doc in docs:
        doc_content = doc.page_content
        for parent_doc in parent_docs:
            if doc_content in parent_doc:
                searched_docs.append(parent_doc)
    
    return searched_docs

In [14]:
searched_parent_docs = search_parent_docs(docs, parent_docs_list)
searched_parent_docs_df = pd.DataFrame({"docs":searched_parent_docs})
print(searched_parent_docs_df.shape)
searched_parent_docs_df = searched_parent_docs_df.drop_duplicates()
searched_parent_docs = searched_parent_docs_df["docs"].tolist()
len(searched_parent_docs)

(20, 1)


15

In [15]:
searched_parent_docs

["휴가 및 복리후생 제도.\n\n법정 휴가 규정.\nCoreFlow는 근로자의 충분한 휴식을 보장하고 일과 삶의 균형을 지원하기 위해 근로기준법 및 관련 법규에 따른 휴가 제도를 운영한다.\n[표]\n종류: 연차 휴가, 근거 법령: 근로기준법 제60조, 일수: 1년 미만 11일, 1년 이상 15일 (가산 휴가 최대 25일), 유급/무급 여부: 유급\n종류: 보상 휴가, 근거 법령: 근로기준법 제57조, 일수: 시간외근로 시간 수만큼 비례 보장, 유급/무급 여부: 유급 (미사용 시 수당 지급)\n종류: 생리 휴가, 근거 법령: 근로기준법 제73조, 일수: 월 1일, 유급/무급 여부: 무급\n종류: 출산전후 휴가, 근거 법령: 근로기준법 제74조, 일수: 출산전후 총 90일 (다태아 120일), 유급/무급 여부: 최초 60일 유급 (다태아 75일)\n종류: 배우자 출산 휴가, 근거 법령: 남녀고용평등법 제18조의2, 일수: 10일, 유급/무급 여부: 유급\n종류: 난임치료 휴가, 근거 법령: 남녀고용평등법 제18조의3, 일수: 연간 3일 이내, 유급/무급 여부: 최초 1일 유급\n종류: 가족돌봄 휴가, 근거 법령: 남녀고용평등법 제22조의2, 일수: 연간 10일 (분할 사용 가능), 유급/무급 여부: 무급\n\n복리후생 규정.\nCoreFlow는 임직원의 경제적 안정, 건강 증진, 그리고 쾌적한 근무 환경 조성을 위해 다양한 복리후생 제도를 제공한다.\n복리후생비: 학자금, 급식 보조비, 체력 단련비, 휴가 보조비 등 사장이 정한 복리후생비를 지급할 수 있다.\n사내 복지 시설: 사내 휴게실, 식당 등을 운영하여 편의를 제공하고, 필요에 따라 휴양 시설 등을 임차하여 직원들이 이용할 수 있도록 지원한다.\n건강 관리: 보건 관리자를 두어 임직원의 건강을 관리하고, 업무상 부상 및 질병에 대한 통계를 작성하여 안전한 근무 환경을 조성한다.\n\n경비 처리 규정.\nCoreFlow는 회사의 자원을 책임감 있게 사용하고, 투명한 회계 관리를 위해 명확한 경비 처리 규

In [16]:
response = requests.post(rerank_url, json={"query":query, "documents":searched_parent_docs}, headers=headers)
response

<Response [200]>

In [17]:
response.json()

{'results': [{'text': "휴가 및 복리후생 제도.\n\n법정 휴가 규정.\nCoreFlow는 근로자의 충분한 휴식을 보장하고 일과 삶의 균형을 지원하기 위해 근로기준법 및 관련 법규에 따른 휴가 제도를 운영한다.\n[표]\n종류: 연차 휴가, 근거 법령: 근로기준법 제60조, 일수: 1년 미만 11일, 1년 이상 15일 (가산 휴가 최대 25일), 유급/무급 여부: 유급\n종류: 보상 휴가, 근거 법령: 근로기준법 제57조, 일수: 시간외근로 시간 수만큼 비례 보장, 유급/무급 여부: 유급 (미사용 시 수당 지급)\n종류: 생리 휴가, 근거 법령: 근로기준법 제73조, 일수: 월 1일, 유급/무급 여부: 무급\n종류: 출산전후 휴가, 근거 법령: 근로기준법 제74조, 일수: 출산전후 총 90일 (다태아 120일), 유급/무급 여부: 최초 60일 유급 (다태아 75일)\n종류: 배우자 출산 휴가, 근거 법령: 남녀고용평등법 제18조의2, 일수: 10일, 유급/무급 여부: 유급\n종류: 난임치료 휴가, 근거 법령: 남녀고용평등법 제18조의3, 일수: 연간 3일 이내, 유급/무급 여부: 최초 1일 유급\n종류: 가족돌봄 휴가, 근거 법령: 남녀고용평등법 제22조의2, 일수: 연간 10일 (분할 사용 가능), 유급/무급 여부: 무급\n\n복리후생 규정.\nCoreFlow는 임직원의 경제적 안정, 건강 증진, 그리고 쾌적한 근무 환경 조성을 위해 다양한 복리후생 제도를 제공한다.\n복리후생비: 학자금, 급식 보조비, 체력 단련비, 휴가 보조비 등 사장이 정한 복리후생비를 지급할 수 있다.\n사내 복지 시설: 사내 휴게실, 식당 등을 운영하여 편의를 제공하고, 필요에 따라 휴양 시설 등을 임차하여 직원들이 이용할 수 있도록 지원한다.\n건강 관리: 보건 관리자를 두어 임직원의 건강을 관리하고, 업무상 부상 및 질병에 대한 통계를 작성하여 안전한 근무 환경을 조성한다.\n\n경비 처리 규정.\nCoreFlow는 회사의 자원을 책임감 있게 사용하고, 투명한 

In [ ]:
best_results = []
for result in response.json()["results"][:5]:
    best_results.append(result["text"])
print(len(best_results))

In [ ]:
def chat_with_ollama(message):
    model=os.getenv("ollama_model")
    url = os.getenv("ollama_url") + "/chat"
    
    messages = []
    messages.append({"role": "system", "content": """
너의 이름은 CoreFlow AI 이고, 너의 역할은 회사 내부 규정에 대한 질문에 답변해 주는 거야. 사용자 프롬프트 포맷은 아래와 같아.

# 질문:
질문 내용
# 첨부된 문서:
문서 내용

첨부된 문서를 활용해서 최대한 자세하게 대답해주되, 모르겠으면 솔직히 모르겠다고 대답해줘.
"""})
    messages.append({"role": "user", "content": f"""
# 질문:
{message}
# 첨부된 문서:
{best_results}
"""})
    # messages.append({"role": "user", "content": message})
    
    payload = {
        "model": model,
        "messages": messages,
        "stream": False,
        "options": {
            "num_ctx": 128000
        }
    }
    
    response = requests.post(url, json=payload).json()["message"]["content"]
    messages.append({"role": "assistant", "content": response})
    return response

In [ ]:
model_response = chat_with_ollama(query)
print(model_response)